# Run Dire Dawa SFINCS Model

## Overview
In this notebook, we will run the Dire Dawa SFINCS hydrodynamic model and visualize the results. The model simulates water surface elevations and flow dynamics in the Dire Dawa region.

## What is SFINCS
SFINCS is a super-fast open-source flood model developed at Deltares. It enables rapid simulation of storm surge, riverine (fluvial) flooding, rainfall-runoff (pluvial), and wave-driven flooding from national, regional to local scales. Designed to run in minutes instead of days, SFINCS supports scenario testing, operational forecasting, and climate adaptation planning worldwide.

**Read the paper introducing SFINCS here** [paper](https://www.sciencedirect.com/science/article/abs/pii/S0378383920304828?via%3Dihub)

## Learning Objectives
- Understand how to set up and run the Dire Dawa SFINCS model.
- Visualize and interpret the model results.

In [2]:
# imports
from pathlib import Path
import os
import matplotlib.pyplot as plt
from hydromt_sfincs import SfincsModel
import rasterio
import pandas as pd
import numpy as np
import xarray as xr
import shutil

In [5]:
# Copy the shared data to the local folder
src = Path("../../data/SFINCS_Dire_Dawa/")
dst = Path("./sfincs_dire_dawa/")

# Create destination folder if missing
dst.mkdir(parents=True, exist_ok=True)

# Copy everything inside src → dst
for item in src.iterdir():
    target = dst / item.name
    if item.is_dir():
        shutil.copytree(item, target, dirs_exist_ok=True)
    else:
        shutil.copy2(item, target)

print("Copied all data into ./data")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '..\\..\\data\\SFINCS_Dire_Dawa'

## Understanding the sfincs model

When making a [SFINCS model](https://sfincs.readthedocs.io/en/latest/example.html), you need to create multiple input files. The model is driven via a main input file named sfincs.inp. This file references a variety of additional input files defining the domain, boundary conditions, topography/elevation, forcing, etc. Required inputs typically include: 
* elevation/topography (grid)
* mask or “active-cell” definitions
* boundary definitions

Optional inputs allow for more detailed configurations (roughness, infiltration, sub-grid tables, observation points). For forcing (i.e. what drives the flood), SFINCS supports various processes: 
* tidal / surge (water-level boundaries)
* rainfall, river discharge
* wind
* pressure
* waves

The model also supports structures human-made features like thin dams, levees (weirs), drainage infrastructure, which can block or divert flow. These are specified via specialized input files (e.g. thin-dam files, weir files) using polylines or geometry definitions. The model setup is supported by various tools. The easiest is the Python-based framework: [HydroMT-SFINCS](https://deltares.github.io/hydromt_sfincs/latest/index.html).

In [8]:
model_dir = r"sfincs_dire_dawa/"

### Input file

The input for SFINCS is supplied using various text (ascii), netcdf and binary files, which are linked through the main input file: sfincs.inp. 

In [9]:
with open(Path(model_dir) / "sfincs.inp") as f:
    print(f.read())

FileNotFoundError: [Errno 2] No such file or directory: 'sfincs_dire_dawa\\sfincs.inp'

In [10]:
import os
os.getcwd()

'C:\\IBF_Workshop\\workshop_export\\Capacity-Building-ICPAC-for-climate-services\\2_IBF_Flood_Damage\\2_Hydrodynamic_Model'

### src file

A simple implementation of discharge points is added to SFINCS, specify values in m^3/s. First specify the locations in ‘sfincs.src’ and then the discharge time-series in ‘sfincs.dis’. Alternatively, you can provide this as netcdf file in the Delft-FEWS format.

The discharge file should be in the following format:
```bash
<src1 x1> <src1 y1>

<src2 x2> <src2 y2>
```

In [ ]:
with open(Path(model_dir) / "sfincs.src") as f:
    print(f.read())

### Preicipitation forcing

In [ ]:
import xarray as xr

ds = xr.open_dataset(Path(model_dir) / "precip_2d.nc")
ds

In [ ]:
ds["Precipitation"].isel(x=5, y=5).plot()

### Grid

In [ ]:
mod = SfincsModel(model_dir, mode="r")

# plot the model layout
fig, ax = mod.plot_basemap(fn_out=None, bmap="sat", figsize=(11, 7))

## Running a SFINCS model

SFINCS can be run on multiple different platforms, both local, HPC and cloud based. The simplest way is to run SFINCS on Windows using a batch-file. The windows executable can be downloaded from https://download.deltares.nl/sfincs. 

Here we installed SFINCS directly on the Ubuntu VM so everyone can run it without having to install it on your own laptop.

* Running the following cell will run the model!

⚠️ **Can take up to 15 minutes and takes a lot of computer resources**

In [ ]:
# ! cd {model_dir} && sfincs

Running is not required, we can look at the log file of output that was produced earlier

In [ ]:
with open(Path(model_dir) / "sfincs_log.txt") as f:
    print(f.read())

### Visualize Results
After running the model, we can visualize the results to analyze the water surface elevations and flow dynamics in the Dire Dawa region.


In [ ]:
mod = SfincsModel(model_dir, mode="r")

In [ ]:
# we can simply read the model results (sfincs_map.nc and sfincs_his.nc) using the read_results method
mod.read_results()
# the following variables have been found
list(mod.results.keys())

### SFINCS Model Output Variables

#### Global Attributes

* **inp**: The full text of the sfincs.inp input file

* **status**: Numeric status code indicating whether the model completed successfully

* **total_runtime**: Total wall-clock runtime of the SFINCS simulation, in seconds

* **average_dt**: The average computational time step actually applied during the simulation, in seconds

#### Grid-Based Map Variables (from sfincs_map.nc)

* **zb**
Bed elevation (digital elevation model) on the SFINCS grid, in metres

* **zs**: Water surface elevation (stage) on the grid for each saved output time step, in metres

* **zsmax**: Maximum water level across the entire simulation for each cell

* **qinf**: Instantaneous infiltration or internal flux per grid cell, in m³/s (volumetric flow)

* **cumprcp**: Cumulative precipitation, integrated over time, typically expressed in metres of water

* **cuminf**: Cumulative infiltration, integrated over time, in metres

#### Point Time-Series Variables (from sfincs_his.nc)

* **point_zb**: Bed elevation at specified observation points

* **point_zs**: Time series of water level at defined observation points, in metres

* **point_qinf**: Time series of infiltration or internal flux at observation points


In [ ]:
# plot the model layout
fig, ax = mod.plot_basemap(fn_out=None, bmap="sat", figsize=(11, 7))

In [ ]:
# Plot time-series:
_ = mod.plot_forcing(fn_out="forcing.png")

In [ ]:
msk = ((mod.results['zsmax'].isel(timemax=2) - mod.results['zb']) > 0.05)
hmax = mod.results['zsmax'].max(dim='timemax')
dmax = hmax - mod.results['zb']

In [ ]:
fig, ax = mod.plot_basemap(fn_out=None, bmap="sat", figsize=(11, 7))
dmax.where(msk).plot(ax=ax, cmap="Blues", vmin=0, vmax=2, zorder=1)

### Plot the timeseries in the cell with max waterdepth

In [ ]:
# Get the index of maximum water level
max_idx = dmax.where(msk).argmax()

# Convert flat index to 2D coordinates
max_coords = np.unravel_index(max_idx, dmax.shape)

d = mod.results['zs'] - mod.results['zb']

# Get the actual lat/lon coordinates using .isel() for integer indexing
max_lat = d.y.isel(y=max_coords[0]).values
max_lon = d.x.isel(x=max_coords[1]).values

print(f"Maximum water level location:")
print(f"Latitude: {max_lat}")
print(f"Longitude: {max_lon}")

d.sel(x=max_lon, y=max_lat, method='nearest').plot()

### Save to tiff raster

In [ ]:
xr.Dataset({"dmax" : dmax}).rio.to_raster("dmax.tif")

## Exercise: Run locally on your machine

In this exercise we will download the sfincs model directory and use the windowls executable to run SFINCS locally on your laptop. 
1. First we need to zip the model directory:

In [ ]:
import os
import zipfile

folder_to_zip = "sfincs_dire_dawa"
zip_filename = "sfincs_dire_dawa.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            filepath = os.path.join(root, file)
            # Put files inside the zip with relative paths
            arcname = os.path.relpath(filepath, folder_to_zip)
            zipf.write(filepath, arcname)

2. The next step is to download the zip. Right click on the created file and click Download.
3. Unzip the directory on your computer.
4. Download pre-compiled executables (e.g. for Windows) under freeware license from the Deltares download portal: https://download.deltares.nl/sfincs ! ITS ALREADY IN THE SOFTWARE FOLDER !
5. Place the SFINCS executable in the same directory
6. Create the following .bat script:
   -  call "c:\..\folder_where_exe_is_located\sfincs.exe">sfincs_log.txt

The file should look like: 

**run.bat**
```bash
call "c:\..\folder_where_exe_is_located\sfincs.exe">sfincs_log.txt
```

### Additional Resources

**Documentation**:
- HydroMT-SFINCS docs: https://deltares.github.io/hydromt_sfincs/
- SFINCS documentation: https://sfincs.readthedocs.io/
- HydroMT core: https://deltares.github.io/hydromt/